# Multi density clustering for evolving datastream
- [Grid](#Grid)
- [Offline Phase](#Offline-Phase)
- [Online Phase](#Online-Phase)
- [Model starts working here](#Working)

In [1]:
__author__='Black D Chase,MR-TLL'
__version__='0.0.1'  

In [2]:
#Imports
import matplotlib.pyplot as plt
import torch
from math import ceil,log2
import log
import random

In [3]:
# Globals - hyperparameters
"""
0 < alpha < 1
lambda > 0
optimal gridGranuality 20 to 30
Data is min normalised b/w [mini,maxi]

minPts is min. required neighbors around a grid (for min point DBSCAN)
"""
lamda = 3
gridGranuality = 25
dimension = 2
      
mini = 0
maxi = 9999
factor = (maxi-mini+1)/gridGranuality
N = factor**dimension

alpha = 2*N*(1-2**(-lamda))

w_cmc = alpha/(N*(1-2**(-lamda)))
minPts = int(2**(dimension*3/4)) 

# DataPoint

In [4]:
class DataPoint:
    def __init__(self,val,t):
        # val is tuple
        self.position = val
        # t is timestamp
        self.t = t
    def __floordiv__(self,divident):
        return tuple([i//divident for i in self.position])
    def print(self):
        print(self.position, self.t)
    

In [5]:
'''
Importing dataset and converting to stream
'''
stream = []
with open('dataset/multiple_dataset_merged_to_stream.txt') as file:
    t = 1
    for line in file:
        line = line[:-1]
        b = line.split('\t')
        batch=[]
        for j in b:
            d = DataPoint(list(map(float, j.split(','))), t)
            batch.append(d)
        stream.append(batch)
        t+=1

# for batch in stream:
#     for item in batch:
#         item.print()
        
stream.append(None)

# Grid
- Region of space which could contain points
- Grid can be:
    - Sporadic Grid $[0]$:
        Region of space which does not contain points
    - Normal Grid $[1]$:
        Region of space which contain points
    - Core Mini Cluster $[2]$:
        Region of space which contains significatn number of points that it could be called as a mini cluster $cmc$

In [6]:
class Grid:
    gType=["Sporadic","Normal","CMC"]
    global alpha,N,lamda,w_cmc,factor,dimension,mini,maxi
    def __init__(self,location,):
        self.location = location
        self.n = 0
        self.t = 0
        self.w = 0
        self.mcd = None
        self.c = None
        self.status = 0
        # hash to be a tuple
        self.storage = []
        self.visited = False
        self.cluster = None
    
    def __hash__(self):
        return hash(self.location)
    
    #""" Might Not need this
    def __eq__(self,other):
        if type(other) == type(self):
            return self.location==other.location
        return self.location==other
    #"""
    def __iter__(self):
        return iter(self.storage)
    
    def updateGS(self,t):
        self.n+=1
        self.w=self.w*2**(-1*lamda*(t - self.t))+1
        self.t=t
        log.debug(f"number of points of {self.n}")
        log.debug(f"weight of the grid {self.w}")
        log.debug(f"updated time {self.t}")
        pass
    
    def calMCD(self):
        ## This is the max distance from mean of all data points on the grid to all the other points on the grid.
        self.calCenter()
        mcd=0
        for points in self:
            assert(len(self.c)==len(points))
            dist=0.0
            for each_dim in points:
                dist+=(self.c - each_dim)**2
            dist**(0.5)
            mcd=max(mcd,dist)

        return mcd
    
    def calCenter(self,time):
        ## time is the current time at the calculation of the center.

        center=[0 for i in self.storage[0]]
        assert(self.w!=0) ## weight must be non-zero for cmc center calc.


        for points in self:
            timestamp=points.t
            weight=2**(-1*self.lamda*(time - timestamp))
            tmplst=[]
            for dims in range(dimension):
                tmplst.append(weight*points[dims])
            for idx in range(dimension):
                center[idx]+=tmplst[idx]

        for idx in range(dimension):
            center[idx]=center[idx]/(self.w)

        return center

    def calRadius(self,time):
        radius=0
        assert(self.w!=0) ## weight must be non-zero for cmc center calc.

        for points in self:
            timestamp=points.t
            weight=2**(-1*self.lamda*(time - timestamp))
            
            
            numerator=(weight*self.getDistance(points,self.c))
            
            radius+=numerator

        radius/=self.w

        return radius
    
    def addPoint(self,point):
        self.storage.append(point)
        self.updateGS(point.t)
        self.updateStatus(point.t)
        if(self.status==2):
            self.__makeitCMC()
            log.debug(f"Status Updated {self.status}")
    
    def getDistance(self,point):
        dist=0.0
        dims=len(point)
        for idx in range(dims):
            dist+=(point[idx] - self.c[idx])**2
        dist=dist**(0.5)

        return dist
    
    def __flush(self):
        """
        Sporadic
        Will be deleted to make room
        """
        self.status = 0 
    
    def __makeitGrid(self):
        """
        Normal
        Will now be a normal grid
        """
        self.status = 1
        
    def __makeitCMC(self,t):
        """
        Core Mini Cluster
        Will now be considered as a mini cluster
        """
        self.calCenter(t)
        self.calMCD(t)
        self.calRadius(t)
        self.status = 2
    
    def __getOWT(self,t):
        OWT = alpha*(1-2**(-lamda*(self.t-t+1)))/(N*(1-2**(-lamda*t)))
        return OWT
    
    def updateStatus(self,t):
        if self.n>1 and self.w>w_cmc:
            self.__makeitCMC(t)
        elif self.w<self.__getOWT(t) or self.n<1:
            self.__flush()
        else:
            self.__makeitGrid()
        return self.status
    
    pass

## Grid Conditions
- Grid weight:
   $$
      W_g(t_c) = \sum_{x \subset g} 2^{-\lambda * (t_c - t_x )}
   $$
   
- Grid weight Update:
   $$
      W_g(t_p,t_c) = 2^{-\lambda * (t_c - t_x )}* w_g(t_p) + 1
   $$
   
- Maximum possible theoretrical Weight of all data points:
   $$
   w_{max} = \frac{1}{1-2^{-\lambda}}
   $$
   
- Time Quantum for Grid and CMC updates 
   $$
   t_{pt} = \frac{log_{2}{\big(\frac{\alpha}{\alpha - N(1-2^{- \lambda})}\big)}}{\lambda}
   $$
   
- Grid->CMC condition:
    $$
      n_g > 1 \text{ and } w_g \ge \frac{\alpha}{N(1-2^{-\lambda})}
    $$

- OWT parameter for GRID and CMC Updates with time  ::

    If $W_g < OWT$ --> remove grid from grid list.
    $$
      OWT(t_c,t_p) = \frac{\alpha(1-2^{-\lambda(t_c - t_p +1)})}{N(1-2^{-\lambda*t_p})}
    $$
    
## CMC Conditions
- Parameters Of CMC when a grid is promoted to CMC:
  $$
  W_{cmc} = W_{g}
  $$
  
  $$
  C_{cmc} = \frac{\sum_{1}^{n} f(t_c - t_i)(p_i)}{w_{cmc}}
  $$
  
  $$
  r_{cmc} = \frac{\sum_{1}^{n} f(t_c - t_i)(distance(p_{ij},c_{cmc}))}{w_{cmc}}
  $$

- If $ W_{cmc} <  \frac{\alpha}{N(1-2^{-\lambda})} $
   Then remove cmc from cmc list

# Cluster
- Combination of one or more mini clusters
- Cluster can be
    - Neighboring grid $[1]$: $N_g$ for $cmc_p$
    - MinPts-nearest-neighbors $[2]$: $N_{sh}(cmc_q)$ for a $cmc_p$
    - Core-neighboring $[3]$:  A core mini-cluster with its MinPts-nearest-neighbors become core-neighboring $N_{core}$ for a $N_{sh}(cmc_p)$
    - Empty $[0]$

In [7]:
class Cluster:
    global alpha,N,lamda,w_cmc,factor,dimension,mini,maxi,minPts
    def __init__(self,name="Unwanted"):
        self._cmcList = {}
        self.status = 0
        self.name = name
        """
        Empty  : 0
        N_g    : 1
        N_sh   : 2
        N_core : 3
        """
    def __eq__(self,other):
        return self.name==other.name
        
    def __hash__(self):
        return hash(self.name)
        
    def __iter__(self):
        return iter(self._cmcList)
    
    def __len__(self):
        return len(self._cmcList)
    
    def has(self,cmc):
        return cmc in self._cmcList.keys()
    
    def getPoint(self,cmc):
        return self._cmcList[cmc]
    
    def addPoint(self,cmc):
        if cmc not in self._cmcList:
            cmc.cluster=self.name
            self._cmcList[cmc]=cmc
    
    def union(self,cluster):
        for cmc in cluster:
            self.addPoint(cluster.getPoint(cmc))
     
    def calDistance(self,x,y):
        dst=0.0
        for idx in range(dimension):
            dst+=(x[idx]-y[idx])**2
        dst=dst**(0.5)

        return dst
        
    def sigma(self,cmc_center,minpts_neighbours):
        mean=self.mu(cmc_center,minpts_neighbours)

        ## minpts_neighbours is a list of all cmc which are kNN to the cmc_center.
        ## Now we will calculate the standard deviation of the distances from these KNN
        distance_list=[]
        length=dimension
        
        for points in minpts_neighbours:
            dst=0.0
            distance_list.append(
                self.calDistance(minpts_neighbours[points],cmc_center)
            )

        sd=0.0
        for dist in distance_list:
            sd+=(dist - mean)**(2)
        sd/=len(distance_list)
        sd=sd**(0.5)

        return sd
    
    def mu(self,cmc_center,minpts_neighbours):
        ## minpts_neighbours is a list of all cmc which are kNN to the cmc_center.
        ## Now we will calculate the mean of the distances from these KNN
        distance_list=[]
        length=dimension
        
        for points in minpts_neighbours:
            dst=0.0
            distance_list.append(
                calDistance(minpts_neighbours[points],cmc_center)
            )
        
        mean=0.0
        for i in distance_list:
            mean+=i
        mean/=len(distance_list)

        return mean


        

# Clusterer

In [8]:
class Clusterer:
    global alpha,N,lamda,w_cmc,factor,dimension,mini,maxi,minPts
    def __init__(self):
        self._gridList={} ## list which containg all the grids as objects.
        self.ng=0 ## no of grids made till now
        pass
    
    def _updateGrid(self,grid,t):
        """
        If a grid is empty, remove it
        #"""
        grid.updateStatus(t)
        log.debug(f"{grid.location} status = {grid.status}")
        log.debug(f"{grid.location} n,w = {grid.n},{grid.w}")
        if grid.status==0:
            sporadic = self._gridList.pop(grid)
            log.debug(f"Popped {sporadic.location}")
            del sporadic
    
    def _findGrid(self, point):
        """
        if a point is (x,y) this it belongs to location (x//factor,y//factor)
        This locataion is the name of grid it will be assigned to
        #"""
        loc = point//factor
        if loc not in self._gridList:
            self._makeGrid(loc)
        return self._gridList[loc]
    
    def _makeGrid(self, loc):
        """
        If grid is not created or deleted to save space recreate it
        #"""
        self._gridList[loc] = Grid(loc)
        log.debug(f"Grid opened at {loc}")
        
    def getCMC(self):
        cmc_S = []
        for grid in self._gridList:
            if self._gridList[grid].status == 2:
                cmc_S.append(self._gridList[grid])
        return cmc_S
    pass

# Online Phase

- CMC-Grid-Neighbourhood
    $$
     {N_g(cmc)} \leftarrow \forall  cmc_p \epsilon {N_g}
    $$
- MinPts nearest Neighbours: In order to determine
- MinPts-nearest-neighbor for the cmc_p, firstly the distance from cmc_p is calculated from all cmc_p-grid-neighborhoods.
- After that, MinPts neighbors are selected with minimum distances and form shorted list neighbors

$$
{N_{sh}(cmc_q)} \leftarrow Minimum(distance((cmc_p),{N_g(cmc_p)}), |N_{sh}(cmc_q)| \ge MinPts
$$

- Core-Neighbouring: A core mini cluster with its MinPts-nearestr-neighbours becomes a core-neighbouring if following conditions are satisfied:

    $$
 {N_{core}} \leftarrow \forall  cmc_q \epsilon {N_{sh}(cmc_p)}
    $$
    
    $$
 \mu(Dist_{cmc_q}) \epsilon [\mu(Dist_{core}) - \sigma(Dist_{core}),\mu(Dist_{core}) + \sigma(Dist_{core})]
    $$
    
    $$
    \mu(Dist_{core}) \leftarrow \mu(distance(N_{sh}(cmc_p),cmc_p)
    $$
    
    $$
    \sigma(Dist_{core}) \leftarrow \sigma(distance(N_{sh}(cmc_p),cmc_p)
    $$
    
    $$
     \mu(Dist_{cmc_q}) \leftarrow \mu(distance(N_{sh}(cmc_q),cmc_q)
    $$
    
    $$
    \sigma(Dist_{cmc_q}) \leftarrow \sigma(distance(N_{sh}(cmc_q),cmc_q)
    $$
    

In [9]:
class Online(Clusterer):
    def __init__(self):
        super(Online,self).__init__()
        self.calPT()
        pass        
    
    def calPT(self):
        """
        Calculating Pruning time
        #"""
        t_pt = log2(alpha/(alpha -N*(1-2**(-lamda))))/lamda
        self.t_pt = ceil(t_pt)
        log.info(f"Pruning time = {self.t_pt}")
        
    def streamData(self,data):
        t=0
        # Time passed
        while(True):
            t+=1
            newPoints = data[t-1]
            """
            []                      : This time step recived no points
            [DataPoint1,DataPoint2] : This time step recived 2 datapoints
            None                    : This marks end of time steps
            #"""
            if newPoints==None:
                break
            for point in newPoints:
                
                grid = self._findGrid(point)
                """
                Grid could be sporadic, normal or cmc
                point will be added 
                #"""
                grid.addPoint(point)
                self._updateGrid(grid,t)
                    
            if t%self.t_pt==0:
                """
                Pruning, after every t_pt timestep
                #"""
                for grid in self._gridList.values():
                    self._updateGrid(grid,t)    
    pass

# Offline Phase

In [10]:
class Offline(Clusterer):
    def __init__(self):
        super(Offline,self).__init__()
        self.clusters = {}
        self.counter=-1
        pass
    
    def uploadCMC(self,cmcList):
        for cmc in cmcList:
            cmc.visited=False
            self._gridList[cmc]=cmc
        log.debug(f"cmc length = {len(self._gridList.keys())}")

    def calDistance(self,x,y):
        dst=0.0
        dim=len(x)
        for idx in range(dim):
            dst+=(x[idx]-y[idx])**2
        dst=dst**(0.5)
        
        log.debug(f"distance {x},{y} = {dst}")
        return dst
    
    def getNeighbours(self,cmc):
        loc = cms.loc
        l=[]
        for i in range(len(loc)): 
            x = list(loc) 
            y = list(loc) 
            x[i]-=1 
            y[i]+=1 
            l.append(tuple(x)) 
            l.append(tuple(y))
        n_g = Cluster()
        n_g.status = 1
        for grid in l:
            if grid in self._gridList:
                n_g.addPoint(self._gridList[grid])
        log.debug(f"{cmc} has {len(n_g)}")
        return n_g
    
    def getName(self):
        self.counter+=1
        return self.counter
    
    def findMPNN(self,n_g,cmc_p,minPts):

        ## n_g is class of all the neighbours , cmc_p is the center of current cmc..
        ## now find minPts nearest neighbours in n_g.
        k_list=[]
        for cmc in n_g:
            k_list.append([self.calDistance(cmc.center,cmc_p),cmc])

        k_list.sort(key = lambda x: x[0])

        clus=Cluster()
        for idx in range(minPts):
            clus.addPoint(self._gridList[k_list[idx][1]])
        
        
        return clus

    def mergeClusters(self):
        keys = list(self._gridList.keys())
        keys = random.shuffle(keys)
        for i in keys:
            cmc_p = self._gridList[i]
            if not cmc_p.visited:
                cmc_p.visited = True
                n_gp = self.getNeighbours(cmc_p)
                
                if len(n_gp)>=minPts:
                    c = Cluster(self.getName())
                    c.addPoint(cmc_p)
                    self.clusters.append(c)
                    n_core = self.findMPNN(n_gp,cmc_p)
                    mu_p,sigma_p = n_core.mu(cmc_p.center,n_core),n_core.sigma(cmc_p.center,n_core)
                    
                    for j in n_core:
                        cmc_q = self._gridList[j]
                        if not cmc_q.visited:
                            cmc_q.visited = True
                            n_gq = self.getNeighbours(cmc_q)
                            
                            if len(n_gq)>=minPts:
                                n_sh = self.findMPNN(n_gq,cmc_q)
                                mu_q,sigma_q = n_sh.mu(cmc_q.center,n_sh),n_sh.sigma(cmc_q.center,n_sh)
                                # Boss's IF condition..DONE it seems
                                mu_cmc_q = mu_q
                                if mu_p-sigma_p <=mu_cmc_q<= mu_p+sigma_p:
                                    n_core.union(n_sh)
                                    mu_p,sigma_p = n_core.mu(cmc_p.center,n_core),n_core.sigma(cmc_p.center,n_core)
                        if cmc_q.cluster is None:
                            c.addPoint(cmc_q)
                else:
                    cmc_p.cluster="NOISE"
                    
    def getClusters(self):
        return self.clusters
    
    pass

# Working

In [11]:
online_component = Online()
online_component.streamData(stream)
cmc_list = online_component.getCMC()
cmc_list

NameError: name 'n' is not defined

In [ ]:
offline_component = Offline()
offline_component.uploadCmc(cmc_list)
offline_component.mergeCluster()
clusters = offline_component.getClusters()
clusters                                                                                                                                                                                                                                                                                          

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

for i in clusters:
    cluster = clusters[i]
    r = random.random()
    b = random.random()
    g = random.random()
    color = (r, g, b)
    
    x=[]
    y=[]
    cmcs = cluster.getCmc()
    for cmc in cmcs:
        for points in cmc.storage:
            x.append(points.position[0])
            y.append(points.position[1])
    ax.scatter(x, y, s=10, c=color, label=cluster.name)
    
plt.legend(loc='upper left');
plt.show()